In [5]:
%pip install json-fix
%pip install clevercsv
%pip install openpyxl
%pip install xlutils
%pip install cleantext
%pip install lxml
%pip install dask
%pip install xlwings
%pip install pyarrow
%pip install pyspark
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
'''
General notes:
- Use pandas for dta that fits into RAM
- For dask dataframes re-indexing is computationally expensive
- xlrd & xlutils & xlwt another method to parse and process excel data
'''
from turtle import back
import pyarrow
import io
from io import StringIO
from __future__ import with_statement
import os
from os import walk
import codecs
import clevercsv
from chardet import detect
from chardet.universaldetector import UniversalDetector
os.environ["NUMEXPR_MAX_THREADS"] = "16"
import csv
import pandas as pd
import dask
import dask.dataframe as ddf
import dask.delayed as dd
from dask.dataframe import read_csv, read_json
from datetime import datetime
import pathlib
from pathlib import Path
from encodings.aliases import aliases
from pyspark.sql import SparkSession
import fastparquet
import json
import yaml
from yaml import CLoader as loader
import json_fix
import re
import ast
import openpyxl
from openpyxl import load_workbook
import xlwings as xw
import xlrd
import xlwt
import xlutils
import cleantext
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import lxml
from lxml.html.clean import Cleaner
main_folder = os.getcwd()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spyro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\spyro\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [36]:
def get_dir_locations():
    '''
    none input
    get cwd and list of folders in it and list of full path to those folders
    3 list outputs
    '''
    current_working_directory = os.getcwd()
    folders = os.listdir(current_working_directory)
    full_path_name_to_folders = []
    for folder in folders:
        full_path_name_to_folders.append(os.path.abspath(folder))
    return current_working_directory, folders, full_path_name_to_folders
 



get_dir_locations()

('c:\\Users\\spyro\\python_file_conversion',
 ['csv_files',
  'csv_to_parquet',
  'html_files',
  'json_files',
  'json_to_parquet',
  'python_file_processing.ipynb',
  'text_files',
  'xl_files',
  'xl_to_parquet'],
 ['c:\\Users\\spyro\\python_file_conversion\\csv_files',
  'c:\\Users\\spyro\\python_file_conversion\\csv_to_parquet',
  'c:\\Users\\spyro\\python_file_conversion\\html_files',
  'c:\\Users\\spyro\\python_file_conversion\\json_files',
  'c:\\Users\\spyro\\python_file_conversion\\json_to_parquet',
  'c:\\Users\\spyro\\python_file_conversion\\python_file_processing.ipynb',
  'c:\\Users\\spyro\\python_file_conversion\\text_files',
  'c:\\Users\\spyro\\python_file_conversion\\xl_files',
  'c:\\Users\\spyro\\python_file_conversion\\xl_to_parquet'])

In [37]:
def list_of_files_in_directory(path: str):
    '''
    1 str input
    extract a list of files from a given directory
    output: list of file names in specified working directory
    '''
    f = []
    for (dirpath, dirnames, filenames) in walk(path):
        f.extend(filenames)
        break
    return f




path = os.path.abspath('csv_files')
list_of_files_in_directory(path)

['annual-enterprise-survey-2021-financial-year-provisional-csv.csv',
 'annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csv',
 'annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csvutf8_convert.csv']

In [38]:
def get_new_file_position_pq(exist_file_path: str):
    '''
    1 str input
    initialise where the new pq file will be posiotioned
    3 str outputs
    '''
    path_exist = pathlib.PurePath(exist_file_path)
    os.chdir(current_working_directory)
    resultant_file_name = path_exist.name + '_to_pq'
    new_file_path = os.path.abspath('csv_to_parquet')
    return path_exist, new_file_path, resultant_file_name

 
    

def csv_file_to_data_frame_to_parquet(local_file_dir: str, parquet_file_dir: str, resultant_pq_file: str) -> None:
    '''
    3 str inputs
    convert the file to pq in initialised new file position
    output: None
    '''    
    dask_df = pd.read_csv(local_file_dir)
    dask_df.to_parquet(parquet_file_dir + '/' +  resultant_pq_file)




current_working_directory = os.getcwd()
try:
    file_path = os.path.abspath('csv_files')
    for root, dirs, files in os.walk(file_path):
        for file in files:
            os.chdir('csv_files')
            new_file_pos = get_new_file_position_pq(Path(file).absolute())
            csv_file_to_data_frame_to_parquet(new_file_pos[0], new_file_pos[1], new_file_pos[2])
finally:
    os.chdir(current_working_directory)

In [13]:
def display_csv_data(csv_file: str) -> None:
    '''
    1 str input
    check the underlying csv file
    output: None
    '''
    fields = []
    rows = []
    with open(csv_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)    
        fields = next(csvreader)
        for row in csvreader:
            rows.append(row)
        print("Total no. of rows: %d"%(csvreader.line_num))
    print('Field names are:' + ', '.join(field for field in fields))
    print('\nFirst 5 rows are:\n')
    for row in rows[:5]:
        for col in row:
            print("%10s"%col,end=" "),
        print('\n')
    csvfile.close()




csv_file = os.path.abspath(r'csv_files\annual-enterprise-survey-2021-financial-year-provisional-csv.csv')
display_csv_data(csv_file)

Total no. of rows: 41716
Field names are:Year, Industry_aggregation_NZSIOC, Industry_code_NZSIOC, Industry_name_NZSIOC, Units, Variable_code, Variable_name, Variable_category, Value, Industry_code_ANZSIC06

First 5 rows are:

      2021    Level 1      99999 All industries Dollars (millions)        H01 Total income Financial performance    757,504 ANZSIC06 divisions A-S (excluding classes K6330, L6711, O7552, O760, O771, O772, S9540, S9601, S9602, and S9603) 

      2021    Level 1      99999 All industries Dollars (millions)        H04 Sales, government funding, grants and subsidies Financial performance    674,890 ANZSIC06 divisions A-S (excluding classes K6330, L6711, O7552, O760, O771, O772, S9540, S9601, S9602, and S9603) 

      2021    Level 1      99999 All industries Dollars (millions)        H05 Interest, dividends and donations Financial performance     49,593 ANZSIC06 divisions A-S (excluding classes K6330, L6711, O7552, O760, O771, O772, S9540, S9601, S9602, and S9603) 

 

In [14]:
def check_encoding(csv_file: str):
    '''
    1 str input
    get the encoding of teh underlying fileinitialise where the new pq file will be posiotioned
    output: str
    '''
    alias_values = set(aliases.values())
    with open(csv_file) as file:
        for encoding in alias_values:
            if file.encoding == encoding:
                file.close()
                return encoding
            else:
                continue




csv_file = r'C:\Users\spyro\python_file_conversion\csv_files\annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csvutf8_convert.csv'
check_encoding(csv_file)

'cp1252'

In [16]:
def find_delimiter_quotechar(csv_file: str):
    '''
    1 str input
    extract delimiter, quotechar and eschapecar from underlying file
    1 outputs: tuple of 2 strs
    '''
    sniffer = csv.Sniffer()
    with open(csv_file) as fp:
        delimiter = sniffer.sniff(fp.read(5000)).delimiter
        quotechar = sniffer.sniff(fp.read(5000)).quotechar
        escapechar = sniffer.sniff(fp.read(5000)).escapechar
    fp.close()
    return delimiter, quotechar, escapechar




csv_file = r'C:\Users\spyro\python_file_conversion\csv_files\annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csv'
find_delimiter_quotechar(csv_file)

(',', '"', None)

In [17]:
def read_csv_data_efficient(csv_file: str) -> None:
    '''
    1 str input
    read csv data efficiently using pre-built libraries
    output: none
    '''
    encoding = check_encoding(csv_file)
    delimiter, quotechar, escapechar = find_delimiter_quotechar(csv_file)
    with open(csv_file, "r", newline="", encoding=encoding) as fp:
        if escapechar == None:
            reader = clevercsv.reader(fp, delimiter=delimiter, quotechar=quotechar, escapechar='')
            rows = list(reader)
        else:
            reader = clevercsv.reader(fp, delimiter=delimiter, quotechar=quotechar, escapechar=escapechar)
            rows = list(reader)   




csv_file = r'C:\Users\spyro\python_file_conversion\csv_files\annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csvutf8_convert.csv'
read_csv_data_efficient(csv_file) 

In [18]:
def get_encoding_type(file: str):
    '''
    1 str input
    exctract encoding in file for use in tests below to check whether utf_8
    output: str
    '''
    with open(file, 'rb') as f:
        rawdata = f.read()
    f.close()
    return detect(rawdata)['encoding']




srcfile = r'C:\Users\spyro\python_file_conversion\csv_files\annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csv'
from_codec = get_encoding_type(srcfile)
try: 
    with open(srcfile, 'r', encoding=from_codec) as f, open(srcfile + 'utf8_convert.csv', 'w', encoding='utf-8') as e:
        text = f.read()
        e.write(text)
    e.close()
except UnicodeDecodeError:
    print('Decode Error')
except UnicodeEncodeError:
    print('Encode Error')

In [19]:
def validateJSON(jsonData: str):
    '''
    1 str input
    check if json file format is consistent
    output: boolean
    '''
    try:
        json.loads(jsonData)
    except ValueError as err:
        return False
    return True




#Tests
InvalidJsonData = """{"name": "jane doe", "salary": 9000, "email": "jane.doe@pynative.com",}""" 
isValid = validateJSON(InvalidJsonData)
print("Given JSON string is Valid", isValid)

validJsonData = """{"name": "jane doe", "salary": 9000, "email": "jane.doe@pynative.com"}"""
isValid = validateJSON(validJsonData)
print("Given JSON string is Valid", isValid)

json_file = os.path.abspath(r'json_files\sample3_ED.json') 
isValid = validateJSON(json_file)
print('Given json string is valid', isValid)


Given JSON string is Valid False
Given JSON string is Valid True
Given json string is valid False


In [46]:
def fix_json_file(file: str) -> None: 
    '''
    1 str input
    correct errors in json file
    output: none
    '''
    f = codecs.open(file, 'r', errors = 'replace')
    text = f.read()
    print(text)
    pattern = '}{'
    print('before editing: ', re.findall(pattern, text))
    newtext = " ".join(text.split())
    print(newtext)
    pattern = '} {'
    print('after editing: ', re.findall(pattern, newtext))
    finaltext = re.sub('} {', '}\n{', newtext)
    print(finaltext)
    f.close
    newfile = file[:-5]+'_ED.json'
    nf = codecs.open(newfile, 'w', errors = 'replace')
    nf.write(finaltext)
    nf.close()
    with open(newfile) as f:
        data = yaml.load(f, Loader= loader)

    with open(file + '_fix_json.json', "w") as f:
        json.dump(data, f)
    os.remove(newfile)
    f.close()




file = os.path.abspath('json_files/sample4.json')
fix_json_file(file)

{
  "people" : [
    {
       "firstName": "Joe",
       "lastName": "Jackson",
       "gender": "male",
       "age": 28,
       "number": "7349282382"
    },
    {
       "firstName": "James",
       "lastName": "Smith",
       "gender": "male",
       "age": 32,
       "number": "5678568567"
    },
    {
       "firstName": "Emily",
       "lastName": "Jones",
       "gender": "female",
       "age": 24,
       "number": "456754675"
    }
  ]
}
before editing:  []
{ "people" : [ { "firstName": "Joe", "lastName": "Jackson", "gender": "male", "age": 28, "number": "7349282382" }, { "firstName": "James", "lastName": "Smith", "gender": "male", "age": 32, "number": "5678568567" }, { "firstName": "Emily", "lastName": "Jones", "gender": "female", "age": 24, "number": "456754675" } ] }
after editing:  []
{ "people" : [ { "firstName": "Joe", "lastName": "Jackson", "gender": "male", "age": 28, "number": "7349282382" }, { "firstName": "James", "lastName": "Smith", "gender": "male", "age": 32, "

In [7]:
'''
Note: Rough work on changing directories and having access to relevant files
'''
os.chdir(main_folder)
print(os.getcwd())
user_folder = os.path.normpath(os.getcwd() + os.sep + os.pardir)
print(user_folder)
print(os.getcwd())
os.chdir('html_files')
print(os.getcwd())
os.chdir(main_folder)
os.chdir('json_files')
print(os.getcwd())
os.chdir(main_folder)
print(os.getcwd())

c:\Users\spyro\python_file_conversion
c:\Users\spyro
c:\Users\spyro\python_file_conversion
c:\Users\spyro\python_file_conversion\html_files
c:\Users\spyro\python_file_conversion\json_files
c:\Users\spyro\python_file_conversion


In [34]:
def json_to_pq(json_file: str, new_file_pos: str, parquet_file_dir: str) -> None:
    '''
    3 str input
    convert json files to pq
    output: none
    '''
    df_json = pd.read_json(json_file)
    print(df_json)
    df_json.to_parquet(parquet_file_dir + '/' +  new_file_pos, engine = 'fastparquet')




#user_folder = os.path.normpath(os.getcwd() + os.sep + os.pardir)
os.chdir(main_folder)
os.chdir('json_files')
json_file = os.path.abspath(r'sample3.json_fix_json.json')
os.chdir(main_folder)
new_file_dir = os.path.abspath('json_to_parquet')
path_exist = pathlib.PurePath(json_file)
new_file_pos = path_exist.name + '_to_pq'
json_to_pq(json_file, new_file_pos, new_file_dir)

c:\Users\spyro\python_file_conversion\json_files\sample3.json_fix_json.json
c:\Users\spyro\python_file_conversion\json_to_parquet
sample3.json_fix_json.json_to_pq
     color value
0      red  #f00
1    green  #0f0
2     blue  #00f
3     cyan  #0ff
4  magenta  #f0f
5   yellow  #ff0
6    black  #000


In [22]:
def excel_get_sheets_rows_data(excel_file: str):
    '''
    1 str input
    get sheets of excel files and rows of data
    output: list of rows with data in excel sheets
    '''
    wb = load_workbook(excel_file, data_only = True)
    sheet_names = wb.sheetnames
    worksheets = [x for x in wb]
    worksheet_rows_all = [list(x.rows) for x in worksheets]   
    new_mother_list = []
    for list_item in worksheet_rows_all:
        new_list = []
        for tuple_object in list_item:
            new_tuple_list = []
            for cell in tuple_object:
                if cell.value == None:
                    cell.value = ''
                    new_tuple_list.append(''.join(list(cell.value)))
                elif type(cell.value) == datetime:
                    new_tuple_list.append(''.join(list(cell.value.strftime('%m/%d/%Y'))))
                else:
                    new_tuple_list.append(''.join(list(str(cell.value))))
            new_list.append(tuple(new_tuple_list))
        new_mother_list.append(new_list)
    return worksheets, new_mother_list




excel_file = os.path.abspath('xl_files\sampledatafoodsales.xlsx')
excel_get_sheets_rows_data(excel_file)

([<Worksheet "Instructions">, <Worksheet "FoodSales">, <Worksheet "MyLinks">],
 [[('', '', ''),
   ('', '', ''),
   ('', '', ''),
   ('', '', ''),
   ('', '', 'Downloaded From'),
   ('', '', 'Sample Data for Excel'),
   ('', '', ''),
   ('', '', 'Related tutorials'),
   ('', '', 'Named Excel Tables'),
   ('', '', 'Data Entry Tips'),
   ('', '', 'More Excel Sample Files'),
   ('', '', ''),
   ('', '', 'Notes'),
   ('', '', 'Fake data to use for Excel testing')],
  [('OrderDate',
    'Region',
    'City',
    'Category',
    'Product',
    'Quantity',
    'UnitPrice',
    'TotalPrice'),
   ('01/01/2020',
    'East',
    'Boston',
    'Bars',
    'Carrot',
    '33',
    '1.7699999999999998',
    '58.41'),
   ('01/04/2020',
    'East',
    'Boston',
    'Crackers',
    'Whole Wheat',
    '87',
    '3.4899999999999998',
    '303.63'),
   ('01/07/2020',
    'West',
    'Los Angeles',
    'Cookies',
    'Chocolate Chip',
    '58',
    '1.8699999999999999',
    '108.46'),
   ('01/10/2020',
   

In [64]:
def excel_dataframe_data_convert(excel_file: str) -> None:
    '''
    1 str input
    get sheets of excel files and rows of data using dataframes in openpyxl
    Note: Same method can be applied with xlwing
    output: None
    '''
    wb = load_workbook(excel_file, data_only = True)
    length = len(list(wb))
    list_of_dataframes = []
    for sheet_index in range(length):
        wb.active = sheet_index
        list_of_dataframes.append(wb.active)
    for dataframe in list_of_dataframes:
        for row in range(0, dataframe.max_row):
            for col in dataframe.iter_cols(1, dataframe.max_column):
                print(col[row].value)




excel_file = os.path.abspath('xl_files\sampledatafoodsales.xlsx')
excel_dataframe_data_convert(excel_file)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
Downloaded From
None
None
Sample Data for Excel
None
None
None
None
None
Related tutorials
None
None
Named Excel Tables
None
None
Data Entry Tips
None
None
More Excel Sample Files
None
None
None
None
None
Notes
None
None
Fake data to use for Excel testing
OrderDate
Region
City
Category
Product
Quantity
UnitPrice
TotalPrice
2020-01-01 00:00:00
East
Boston
Bars
Carrot
33
1.7699999999999998
58.41
2020-01-04 00:00:00
East
Boston
Crackers
Whole Wheat
87
3.4899999999999998
303.63
2020-01-07 00:00:00
West
Los Angeles
Cookies
Chocolate Chip
58
1.8699999999999999
108.46
2020-01-10 00:00:00
East
New York
Cookies
Chocolate Chip
82
1.87
153.34
2020-01-13 00:00:00
East
Boston
Cookies
Arrowroot
38
2.18
82.84
2020-01-16 00:00:00
East
Boston
Bars
Carrot
54
1.77
95.58
2020-01-19 00:00:00
East
Boston
Crackers
Whole Wheat
149
3.4899999999999998
520.01
2020-01-22 00:00:00
West
Los Angeles
Bars
Carrot
51
1.77
90.27
2020-01-25 00:00:00
Ea

In [4]:
def excel_to_pq(excel_file: str, new_file_pos: str, parquet_file_dir: str) -> None:
    '''
    3 str input
    convert excel files to pq
    output: none
    '''
    all_sheets_df = pd.read_excel(excel_file, na_values = "Missing",
                                                     sheet_name = None)
    for key in all_sheets_df:                                                   
        all_sheets_df[key].to_parquet(parquet_file_dir + f'/{key}' +  new_file_pos)




excel_file = os.path.abspath('xl_files\sampledatainsurance.xlsx')
new_file_dir = os.path.abspath('xl_to_parquet')
path_exist = pathlib.PurePath(excel_file)
new_file_pos = path_exist.name + '_to_pq'
excel_to_pq(excel_file, new_file_pos, new_file_dir)

In [7]:
def text_data_processing(text_file: str):
    '''
    1 str input
    clean text data from non-alphanumeric characters and return lists of the words and their tags (nouns, verbs etc)
    Note: Similar method can be applied with cleantext.clen and cleantext.clean_words 
    output: 2 lists
    '''
    with open(text_file) as f:
        lines = f.readlines()
        tokens = []
        tokens_pos_tag = []
        stop = stopwords.words('english')
        for line in lines:
            line = line.lower()
            line = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", line)
            line = " ".join(word for word in line.split() if word not in (stop))
            token = nltk.word_tokenize(line)
            tokens.append(tuple(token))
            pos = nltk.pos_tag(token)
            tokens_pos_tag.append(tuple(pos))
        f.close()
    return tokens, tokens_pos_tag
       



text_file = os.path.abspath('text_files\iso_8859_sample.txt')
text_data_processing(text_file)

([('following', 'graphical', 'noncontrol', 'characters', 'defined'),
  ('iso', '88591', '1987', 'descriptions', 'words', 'arent', 'helpful'),
  ('theyre', 'best', 'text', 'graphics', 'file', 'illustrating'),
  ('character', 'set', 'available', 'archive'),
  ('file',),
  (),
  ('hex', 'description', 'hex', 'description'),
  (),
  ('20', 'space'),
  ('21', 'exclamation', 'mark', 'a1', 'inverted', 'exclamation', 'mark'),
  ('22', 'quotation', 'mark', 'a2', 'cent', 'sign'),
  ('23', 'number', 'sign', 'a3', 'pound', 'sign'),
  ('24', 'dollar', 'sign', 'a4', 'currency', 'sign'),
  ('25', 'percent', 'sign', 'a5', 'yen', 'sign'),
  ('26', 'ampersand', 'a6', 'broken', 'bar'),
  ('27', 'apostrophe', 'a7', 'section', 'sign'),
  ('28', 'left', 'parenthesis', 'a8', 'diaeresis'),
  ('29', 'right', 'parenthesis', 'a9', 'copyright', 'sign'),
  ('2a', 'asterisk', 'aa', 'feminine', 'ordinal', 'indicator'),
  ('2b',
   'plus',
   'sign',
   'ab',
   'leftpointing',
   'double',
   'angle',
   'quotation'

In [3]:
def html_cleaner(html_file: str):
    '''
    1 str input
    clean html files using Cleaner pre-developed class. Cleaner arguments can be adjusted accordingly
    output: 1 object cleaner
    '''
    cleaner = Cleaner(page_structure=True,
                  meta=True,
                  embedded=True,
                  links=True,
                  style=True,
                  processing_instructions=True,
                  inline_style=True,
                  scripts=True,
                  javascript=True,
                  comments=True,
                  frames=True,
                  forms=True,
                  annoying_tags=True,
                  remove_unknown_tags=True,
                  safe_attrs_only=True,
                  safe_attrs=frozenset(['src','color', 'href', 'title', 'class', 'name', 'id']),
                  remove_tags=('span', 'font', 'div')
                  )
    return cleaner.clean_html(html_file)




def display_clean_html(html_file: str) -> None:
    '''
    1 str input
    write cleaned html file to new path
    output: none
    '''
    with open(html_file) as fin, open(html_file + 'cleaned_converted.html', 'w') as new:
        cleaning = html_cleaner(fin.read())
        print(cleaning)
        new.write(cleaning)
        fin.close()
        new.close()




html_file = os.path.abspath(r'html_files/sample1 (1).html')
display_clean_html(html_file)

<div>






















	Sample HTML 1

<base href="https://filesamples.com/samples/code/html/sample1.html">






<body>
	Sample HTML 1
	<h1>Minime vero, inquit ille, consentit.</h1>

	<p>Lorem ipsum dolor sit amet, consectetur adipiscing elit. Inscite autem medicinae et gubernationis ultimum cum ultimo sapientiae comparatur. Cur igitur, cum de re conveniat, non malumus usitate loqui? </p>

	<ol>
		<li>Si qua in iis corrigere voluit, deteriora fecit.</li>
		<li>At quicum ioca seria, ut dicitur, quicum arcana, quicum occulta omnia?</li>
		<li>An dolor longissimus quisque miserrimus, voluptatem non optabiliorem diuturnitas facit?</li>
		<li>Multoque hoc melius nos veriusque quam Stoici.</li>
		<li>Stuprata per vim Lucretia a regis filio testata civis se ipsa interemit.</li>
		<li>Ego vero isti, inquam, permitto.</li>
	</ol>


	<p>Graecum enim hunc versum nostis omnes-: Suavis laborum est praeteritorum memoria. Qui enim existimabit posse se miserum esse beatus non erit. Si qua in iis